In [5]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import gensim
from collections import defaultdict
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from dataLoad import testValidLoad, testLoad
from metric import ndcg_calculator, hit_at_k

path= "../dataset/"
test, final_submission, test_user_label = testLoad(path)

## word2Vec :: 
연속된 item 조회 기록을 통해, 연속성을 가지고 조회되는 item의 잠재요인을 찾아보려는 시도

In [ ]:
""" # 목적

1. candidate - similar items
    ㄴ  1. test dataframe을 가져와 유저별 가장 마지막 조회 아이템을 찾는다.
        2. word2Vec 모델을 사용해 10개씩 새로운 candidate dataframe에 user:item 형태로 row 추가.
        3. 추천 항목으로 쓰기 위해, dict 형태로도 return한다.
            candidate 뿐만 아니라, 추천 항목이 부족할 때 (lgbm pred에 10개가 안 채워질 때) 패딩 용도로도 쓸 수 있다.
        
2. feature engineering - item cluster
    ㄴ 시간 부족으로 중단


"""

In [6]:
sentences_df = test.groupby("user")["item_id"].agg(lambda x: ", ".join(x.astype(str))).reset_index()

In [7]:
sentences = sentences_df["item_id"].to_list()

In [30]:
sentences

['766598, 65273, 580437, 580437, 773271, 228561, 18748, 6725, 99247, 156185, 171244, 760540, 396362, 453078, 784005, 369709, 888904, 784458, 6821, 944280',
 '183462, 374145, 374145, 44563, 44563, 204302, 110301, 793635, 936650, 293981, 582874, 738005, 563302, 31640, 293913, 19776, 597318, 839005, 689033, 148090',
 '108368, 41459, 667880, 667880, 869425, 869425, 932949, 832922, 28643, 710805, 205108, 911302, 27526, 561965, 716642, 247703, 180739, 775306, 852611, 267761',
 '520146, 670721, 962828, 560874, 181147, 544492, 614476, 126927, 377829, 949905, 736959, 401691, 20626, 613959, 812984, 300385, 470634, 739789, 248163, 375779',
 '577279, 374594, 99709, 382452, 747701, 717701, 717701, 562399, 794553, 168299, 486989, 944049, 895860, 731440, 5053, 374586, 533344, 563744, 786522, 641013',
 '154566, 183998, 654574, 896230, 78288, 539528, 899651, 801066, 51920, 197684, 738572, 875255, 521132, 446800, 34132, 514567, 489126',
 '73722, 439630, 218486, 50740, 307522, 184767, 416119, 807300, 313

## test model 

In [31]:
%%time
w2vec = Word2Vec(sentences=sentences, vector_size=200, min_count=1, workers=4)

CPU times: total: 2min 1s
Wall time: 51.6 s


In [11]:
w2vec.save("../output/test_word2vec.model")

In [32]:
result

[('1', 0.8246535062789917),
 ('7', 0.8142736554145813),
 ('6', 0.8136557936668396),
 ('2', 0.808832049369812),
 ('9', 0.758715033531189),
 ('3', 0.7502750158309937),
 ('4', 0.7378901243209839),
 ('0', 0.7310563325881958),
 ('5', 0.7151944041252136),
 (',', -0.1370769888162613)]

In [35]:
word = 2
result = w2vec.wv.most_similar(positive=[word], topn=10)
for word, similarity in result:
    print(f"Word: {word}, Similarity: {similarity}")

Word: 2, Similarity: 0.8295935392379761
Word: 9, Similarity: 0.8078092336654663
Word: 6, Similarity: 0.769846498966217
Word: 4, Similarity: 0.7636667490005493
Word: 8, Similarity: 0.7600404024124146
Word: 7, Similarity: 0.7596796751022339
Word: 3, Similarity: 0.7550446391105652
Word: 0, Similarity: 0.7472016215324402
Word: 5, Similarity: 0.7104031443595886
Word: ,, Similarity: -0.029944563284516335


In [14]:
# 마지막 로그 가져와서 df로 만드는 코드 
def take_lastClick(df):
    lastClick_df = df.sort_values(by="timestamp", ascending=False)
    lastClick_df = lastClick_df.drop_duplicates(subset="user", keep="first") 
    lastClick_df = lastClick_df[["user","item_id"]]
    return lastClick_df

def make_Word2Vec_dict(lastClick_df):
    Word2Vec_dict= defaultdict(list)
    for index, row in lastClick_df.iterrows():
        user = row["user"]
        item = row["item_id"]
        result = w2vec.wv.most_similar(positive=[item], topn=10)
        for word, similarity in result:
            Word2Vec_dict[user].append(word)
    return Word2Vec_dict    

def Word2Vec_candidate(Word2Vec_dict):
    Word2Vec_candidate = pd.DataFrame()
    num_rows = sum(len(v) for v in Word2Vec_dict.values())
    Word2Vec_candidate = pd.DataFrame(index=range(num_rows), columns=["user", "item_id"])
    row_idx = 0
    for key, values in Word2Vec_dict.items():
        for value in values:
            Word2Vec_candidate.loc[row_idx] = [key, value]
            row_idx += 1
    return Word2Vec_candidate

In [15]:
lastClick_df = take_lastClick(test)
Word2Vec_dict = make_Word2Vec_dict(lastClick_df)

KeyError: "Key '571805' not present in vocabulary"

In [ ]:
# Word2Vec_candidate = pd.DataFrame()

# num_rows = sum(len(v) for v in Word2Vec_dict.values())
# Word2Vec_candidate = pd.DataFrame(index=range(num_rows), columns=["user", "item_id"])
# row_idx = 0
# for key, values in Word2Vec_dict.items():
#     for value in values:
#         Word2Vec_candidate.loc[row_idx] = [key, value]
#         row_idx += 1

In [55]:
from dataLoad import testLoad
path= "../dataset/"
fe_test, final_submission, test_user_label = testLoad(path)

In [48]:
test = test.groupby("user")["item_id"].agg(lambda x: ", ".join(x.astype(str))).reset_index()
sentences = test["item_id"].to_list()

In [51]:
w2vec = Word2Vec(sentences=sentences, vector_size=32, min_count=1, workers=4)

In [ ]:
word = 315929
result = w2vec.wv.most_similar(positive=[word], topn=10)
for word, similarity in result:
    print(f"Word: {word}, Similarity: {similarity}")

In [56]:
lastClick_df2 = take_lastClick(test)

In [57]:
Word2Vec_dict2 = make_Word2Vec_dict(lastClick_df2)

KeyError: "Key '128050' not present"

In [ ]:
import pickle
pickle.dump(Word2Vec_dict, open("../output/test_Word2Vec_dict.pkl", "wb"))
# Word2Vec_candidate.to_parquet("../output/valid_Word2Vec_candidate.parquet", index=False)

pickle.dump(Word2Vec_dict2, open("../output/test_Word2Vec_dict.pkl", "wb"))
# Word2Vec_candidate2.to_parquet("../output/test_Word2Vec_candidate.parquet", index=False)